In [ ]:
import numpy as np
import pandas as pd
import torch
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from rank_bm25 import BM25Okapi
from multiprocessing import Pool,Lock
import tqdm
import catboost
import operator
from sklearn.feature_extraction.text import TfidfVectorizer
import lightgbm as lg
from gensim.models import FastText

In [ ]:
#data 
queries = {}
with open('./normal_queries.txt') as fin:
    lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if line[0] == '':
            line.pop(0)
        queries[line[0]] = line[1]

        titles= {}
with open("./normal_titles.txt" ,'r', encoding='utf-8') as fin:
    for line in fin.readlines():
        line=line.split('\t')
        titles[line[0]]= line[1][:-1]

train=[]
train_titles = []
train_queries = []
trq = set()
with open("./train.marks.tsv/train.marks.tsv", 'r') as fin:
    for line in fin.readlines():
        line=line[:-1].split('\t')
        if line[1] in titles.keys():
            train.append([line[0],line[1], line[2]])
            train_titles.append([line[1], titles[line[1]]])
            if line[0] not in trq:
                train_queries.append([line[0], queries[line[0]]])
                trq.add(line[0])

                
test=[]
test_titles = []
test_queries = []
teq = set()
with open("./sample.csv/sample.csv", 'r') as fin:
    fin.readline()
    for line in fin.readlines():
        line=line[:-1].split(',')
        if line[1] in titles.keys():
            test.append([line[0],line[1],-1])
            test_titles.append([line[1], titles[line[1]]])
            if line[0] not in teq:
                test_queries.append([line[0], queries[line[0]]])
                teq.add(line[0])

alq = list(train) + list(test)
dctqs = {}
for q, t, n in alq:
    if q not in dctqs.keys():
        dctqs[q] = []
    dctqs[q].append(t)
alqueries = train_queries + test_queries
id_q = {}
for el in alqueries:
    id_q[el[0]] = el[1]   

altits = {}
for el in train_titles + test_titles:
    altits[el[0]] = el[1]

In [ ]:
import tensorflow as tf
from absl import logging
import sentencepiece
import tensorflow_text
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

outdata = "features/luse"
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

def cos(sp1,sp2):
    #sp1 = sp1.todense()
    #sp2 = sp2.todense()
    return np.float64(np.dot(sp1,sp2.T)/np.linalg.norm(sp1)/np.linalg.norm(sp2))

import time
def getsimuse(qid):
    raw_tits = {}
    global id_q
    global altits
    global embed
    #t1 = time.time()
    for dc in dctqs[qid]:
        try:
            raw_tits[dc] = altits[dc]
        except:
            print(qid, dc)
    q = embed([id_q[qid]]).numpy()
    embs = embed(list(raw_tits.values())).numpy()
    #t2 = time.time()
    #print(t2 - t1)
    results = []
    with open('features/luse/{}.txt'.format(qid),'w') as fout:
        for i,doc in enumerate(raw_tits.keys()):
            fout.write(str(qid) + '\t' +str(doc) + '\t' + str(cos(q,embs[i])) + '\n')
    #print(time.time() - t2)
    del embs
    del q
    del raw_tits
    return results

for q in tqdm.tqdm(list(dctqs.keys())[2891:]):
    r = getsimuse(q)
lfiles = os.listdir(outdata)
with open('features/luse.txt','w') as fout:
    for file in lfiles:
        with open(outdata+file) as fin:
            line = fin.readline()
            while line != '':
                fout.write(line)
                line = fin.readline()

In [ ]:
embed_altits = {}

for k in tqdm.tqdm(altits.keys()):
    embed_altits[k] = embed(altits[k]).numpy()

In [ ]:
import pickle
pickle_obj = open("embed_altits.pickle","wb")
pickle.dump(embed_altits, pickle_obj)
pickle_obj.close()

In [ ]:
pickle_obj = open("embed_altits.pickle", "rb")
embed_altits_pickle = pickle.load(pickle_obj)
pickle_obj.close()
embed_altits_pickle

In [ ]:
import tensorflow as tf
from absl import logging
import sentencepiece
import tensorflow_text
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

outdata = "features/luse/"
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

def cos(sp1,sp2):
    return np.float64(np.dot(sp1,sp2.T)/np.linalg.norm(sp1)/np.linalg.norm(sp2))

import time
def getsimuse(qid):
    raw_tits = {}
    global id_q
    global altits
    global embed
    global embed_altits
    embs = []
    for dc in dctqs[qid]:
        try:
            raw_tits[dc] = altits[dc]
            embs.append(embed_altits[dc])
        except:
            print(qid, dc)
    q = embed([id_q[qid]]).numpy()
    results = []
    with open('features/luse/{}.txt'.format(qid),'w') as fout:
        for i,doc in enumerate(raw_tits.keys()):
            fout.write(str(qid) + '\t' +str(doc) + '\t' + str(cos(q,embs[i])) + '\n')
    del embs
    del q
    del raw_tits
    return results

for q in tqdm.tqdm(list(dctqs.keys())):
    r = getsimuse(q)
lfiles = os.listdir(outdata)
with open('features/luse.txt','w') as fout:
    for file in lfiles:
        with open(outdata+file) as fin:
            line = fin.readline()
            while line != '':
                fout.write(line)
                line = fin.readline()

In [ ]:
dtitles = [TaggedDocument(doc, [i]) for i, doc in train_titles]
dqueries = [TaggedDocument(doc, [i]) for i, doc in train_queries]
dstitles = [TaggedDocument(doc, [i]) for i, doc in test_titles]
dsqueries = [TaggedDocument(doc, [i]) for i, doc in test_queries]
#docvec similarity on queries
from nltk.tokenize import word_tokenize
aq = [TaggedDocument(doc,[i]) for i, doc in alqueries]
dv = Doc2Vec(aq, vector_size=128, window=2, min_count=1, workers=4)
max_epochs = 100
alpha = 0.0025
dv = Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, sample = 0)
dv.build_vocab(aq)
for epoch in range(max_epochs):
    dv.train(aq,
    total_examples=dv.corpus_count,
    epochs=dv.iter)
    # decrease the learning rate
    dv.alpha -= 0.0002
    # fix the learning rate, no decay
    dv.min_alpha = dv.alpha

In [ ]:
dvecs = {}
for el in alqueries:
    dvecs[el[0]] = dv.docvecs[el[0]]

In [ ]:
def cos(sp1,sp2):
    return np.float64(np.dot(sp1,sp2.T)/np.linalg.norm(sp1)/np.linalg.norm(sp2))

In [ ]:
simdoc = {}
def getsim(key):
    global dvecs
    tmp = {}
    for tkey in dvecs.keys():
        if tkey != key:
            tmp[tkey] = cos(dvecs[key],dvecs[tkey])
    sortres = sorted(tmp.items(), key = operator.itemgetter(1), reverse = True)
    simdoc[key] = []
    res = []
    for i in range(20):
        res.append(sortres[i][0])
    return [key,res]

sims = {}
for key in tqdm.tqdm(dvecs.keys()):
    sims[key] = getsim(key)[1]    


In [ ]:
with open('features/prevsimdoc.txt','w') as fout:
    for key in sims.keys():
        fout.write(key+'\t')
        for el in sims[key]:
            fout.write(el+' ')
        fout.write('\n')


In [ ]:
#multiliggual USE
import os
import tensorflow as tf
from absl import logging
import sentencepiece
import tensorflow_text
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns


embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [ ]:
embed_altits_multiligual = {}

for k in tqdm.tqdm(altits.keys()):
    embed_altits_multiligual[k] = embed(altits[k]).numpy()

In [ ]:
import pickle
pickle_obj = open("embed_altits_multiligual.pickle","wb")
pickle.dump(embed_altits_multiligual, pickle_obj)
pickle_obj.close()

In [ ]:
pickle_obj = open("embed_altits_multiligual.pickle", "rb")
embed_altits_multiligual_pickle = pickle.load(pickle_obj)
pickle_obj.close()
embed_altits_multiligual_pickle

In [ ]:
def cos(sp1,sp2):
    return np.float64(np.dot(sp1,sp2.T)/np.linalg.norm(sp1)/np.linalg.norm(sp2))

import time
def getsimuse(qid):
    raw_tits = {}
    global id_q
    global altits
    global embed
    global embed_altits_multiligual
    embs = []
    for dc in dctqs[qid]:
        try:
            raw_tits[dc] = altits[dc]
            embs.append(embed_altits_multiligual[dc])
        except:
            print(qid, dc)
    q = embed([id_q[qid]]).numpy()
    results = []
    with open('./features/muse/{}.txt'.format(qid),'w') as fout:
        for i,doc in enumerate(raw_tits.keys()):
            fout.write(str(qid) + '\t' +str(doc) + '\t' + str(cos(q,embs[i])) + '\n')
    del embs
    del q
    del raw_tits
    return results
for q in tqdm.tqdm(dctqs.keys()):
    r = getsimuse(q)
mfiles = os.listdir('./features/muse/')
with open('./features/muse.txt','w') as fout:
    for file in mfiles:
        with open('./features/muse/'+file) as fin:
            line = fin.readline()
            while line != '':
                fout.write(line)
                line = fin.readline()

In [ ]:
#docvec similarity on queries
from nltk.tokenize import word_tokenize
#dq = [TaggedDocument(doc,[i]) for i, doc in train_titles + test_titles]

lst = []
for key in dctqs.keys():    
    for el in dctqs[key]:
        lst.append([el,altits[el]])
    
dq = [TaggedDocument(doc,[i]) for i, doc in lst]
for key in dctqs.keys():
    dq.append(TaggedDocument(id_q[key],[1111111]))

In [ ]:
dctqs

In [ ]:
len(dq)

In [ ]:
len(train_titles + test_titles)

In [ ]:
dv = Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, sample = 0)

In [ ]:
max_epochs = 100
alpha = 0.0025
dv = Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, sample = 0)
dv.build_vocab(dq)

In [ ]:
for epoch in tqdm.tqdm(range(max_epochs)):
        dv.train(dq,
        total_examples=dv.corpus_count,
        epochs=dv.iter)
        dv.alpha -= 0.0002
        dv.min_alpha = dv.alpha

In [ ]:
import pickle
pickle_obj = open("dv.pickle","wb")
pickle.dump(dv, pickle_obj)
pickle_obj.close()

In [ ]:
def cos(sp1,sp2):
    return np.float64(np.dot(sp1,sp2.T)/np.linalg.norm(sp1)/np.linalg.norm(sp2))


In [ ]:
#docvec similarity on titles
def docdst(key):
    lst = []
    global dctqs
    global altits
    global id_q
    global dv
    with open('./features/dv/{}.txt'.format(key),'w', encoding = 'utf-8') as fout:
          for el in dctqs[key]:
                fout.write(str(key) + '\t' + str(el) + '\t' + str(cos(dv.docvecs[1111111],dv.docvecs[el]))+'\n')
    return key

In [ ]:
for key in tqdm.tqdm(dctqs.keys()):
    docdst(key)

In [ ]:
import os
mfiles = os.listdir('./features/dv/')

with open('./features/dvs.txt','w', encoding = 'utf-8') as fout:
    for file in mfiles:
        if file.endswith(".txt"):
            with open('./features/dv/'+file, "r", encoding = 'utf-8') as fin:
                line = fin.readline()
                while line != '':
                    fout.write(line)
                    line = fin.readline()

In [ ]:
#fasttext trained
import fasttext.util
import fasttext
from gensim.models import FastText
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
model = FastText(min_count=1, size = 10, workers=8, negative=5, iter=10, min_n=2, max_n=10)
def getsim(q):
    global dctqs
    global id_q
    global altits
    raw = {}
    model = FastText(min_count=1, size = 10, workers=2, negative=3, iter=10, min_n=2, max_n=9)
    for doc in dctqs[q]:
        try:
            raw[doc] = word_tokenize(altits[doc])
        except:
            continue
    raw['-1'] = word_tokenize(id_q[q])
    model.build_vocab(raw.values())
    model.train(raw.values(), total_examples=model.corpus_count, epochs=10)
    with open('./features/trft/{}.txt'.format(q),'w') as fout:
        for outdoc in raw.keys():
            if outdoc != '-1':
                fout.write(str(outdoc)+ '\t'+str(cos(model.wv[altits[outdoc].replace(' ','_')], model.wv[id_q[q].replace(' ','_')])) + '\n')
    return




In [ ]:
for key in tqdm.tqdm(dctqs.keys()):
    getsim(key)

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
import os
import operator
from multiprocessing import Pool,Lock         
#data 
queries = {}
with open('./normal_queries.txt') as fin:
    lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if line[0] == '':
            line.pop(0)
        queries[line[0]] = line[1]

        titles= {}
with open("./normal_titles.txt" ,'r', encoding='utf-8') as fin:
    for line in fin.readlines():
        line=line.split('\t')
        titles[line[0]]= line[1][:-1]

train=[]
train_titles = []
train_queries = []
trq = set()
with open("./train.marks.tsv/train.marks.tsv", 'r') as fin:
    for line in fin.readlines():
        line=line[:-1].split('\t')
        if line[1] in titles.keys():
            train.append([line[0],line[1], line[2]])
            train_titles.append([line[1], titles[line[1]]])
            if line[0] not in trq:
                train_queries.append([line[0], queries[line[0]]])
                trq.add(line[0])

                
test=[]
test_titles = []
test_queries = []
teq = set()
with open("./sample.csv/sample.csv", 'r') as fin:
    fin.readline()
    for line in fin.readlines():
        line=line[:-1].split(',')
        if line[1] in titles.keys():
            test.append([line[0],line[1],-1])
            test_titles.append([line[1], titles[line[1]]])
            if line[0] not in teq:
                test_queries.append([line[0], queries[line[0]]])
                teq.add(line[0])

alq = list(train) + list(test)
dctqs = {}
for q, t, n in alq:
    if q not in dctqs.keys():
        dctqs[q] = []
    dctqs[q].append(t)
alqueries = train_queries + test_queries
id_q = {}
for el in alqueries:
    id_q[el[0]] = el[1]   

altits = {}
for el in train_titles + test_titles:
    altits[el[0]] = el[1]
def cos(sp1,sp2):
    sp1 = sp1.todense()
    sp2 = sp2.todense()
    try:
        res = np.float64(np.dot(sp1,sp2.T)/np.linalg.norm(sp1)/np.linalg.norm(sp2))
    except:
        res = None
    return res
try:
    os.mkdir('./features/ttfsim')
except:
    n = None
def ptfidfsim(q):
    global dctqs
    global id_q
    global altits
    tf = TfidfVectorizer(analyzer = 'char', ngram_range=(3,9))
    raw = {}
    for doc in dctqs[q]:
        try:
            raw[doc] = altits[doc]
        except:
            continue
    raw['-1'] = id_q[q]
    vecs = tf.fit_transform(raw.values())
    with open('./features/ttfsim/{}.txt'.format(q),'w') as fout:
        for i,outdoc in enumerate(raw.keys()):
            if outdoc != '-1':
                fout.write(str(outdoc)+ '\t'+str(cos(vecs[i],vecs[-1])) + '\n')
    return

def pLtfidfsim(q):
    global dctqs
    global id_q
    global altits
    tf = TfidfVectorizer(analyzer = 'char', ngram_range=(1,3))
    raw = {}
    for doc in dctqs[q]:
        try:
            raw[doc] = altits[doc]
        except:
            continue
    raw['-1'] = id_q[q]
    vecs = tf.fit_transform(raw.values())
    with open('./features/ttfsim/{}.txt'.format(q),'w') as fout:
        for i,outdoc in enumerate(raw.keys()):
            if outdoc != '-1':
                fout.write(str(outdoc)+ '\t'+str(cos(vecs[i],vecs[-1])) + '\n')
    return


In [ ]:
for q,t in tqdm.tqdm(alqueries):
    ptfidfsim(q)


In [ ]:
import os
from multiprocessing import Pool
import numpy as np
from nltk.tokenize import word_tokenize
#data 
queries = {}
with open('./normal_queries.txt', encoding='utf-8') as fin:
    lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if line[0] == '':
            line.pop(0)
        queries[line[0]] = line[1]

        titles= {}
with open("./normal_titles.txt" ,'r', encoding='utf-8') as fin:
    for line in fin.readlines():
        line=line.split('\t')
        titles[line[0]]= line[1][:-1]

train=[]
train_titles = []
train_queries = []
trq = set()
with open("./train.marks.tsv/train.marks.tsv", 'r', encoding='utf-8') as fin:
    for line in fin.readlines():
        line=line[:-1].split('\t')
        if line[1] in titles.keys():
            train.append([line[0],line[1], line[2]])
            train_titles.append([line[1], titles[line[1]]])
            if line[0] not in trq:
                train_queries.append([line[0], queries[line[0]]])
                trq.add(line[0])

                
test=[]
test_titles = []
test_queries = []
teq = set()
with open("./sample.csv/sample.csv", 'r', encoding='utf-8') as fin:
    fin.readline()
    for line in fin.readlines():
        line=line[:-1].split(',')
        if line[1] in titles.keys():
            test.append([line[0],line[1],-1])
            test_titles.append([line[1], titles[line[1]]])
            if line[0] not in teq:
                test_queries.append([line[0], queries[line[0]]])
                teq.add(line[0])

alq = list(train) + list(test)
dctqs = {}
for q, t, n in alq:
    if q not in dctqs.keys():
        dctqs[q] = []
    dctqs[q].append(t)
alqueries = train_queries + test_queries
id_q = {}
for el in alqueries:
    id_q[el[0]] = el[1]   

altits = {}
for el in train_titles + test_titles:
    altits[el[0]] = el[1]

outdata = './features/pasT/'
def getpscore(qid):
    global dctqs
    global id_q
    global altits
    texts = {}
    idfdict = {}
    for doc in dctqs[qid]:
        try:
            texts[doc] = word_tokenize(altits[doc])
            for w in texts[doc]:
                if w not in idfdict.keys():
                    idfdict[w] = 0
                idfdict[w] += 1
        except:
            continue
    for key in idfdict.keys():
        idfdict[key] = len(texts) / idfdict[key]
    
    query = word_tokenize(id_q[qid])
    qset = set(query)
    with open(outdata+qid+'.txt','w') as fout:
        for key in texts.keys():
            ltext = texts[key]
            pas = 0
            for i in range(len(ltext) - 2):
                scut = set(ltext[i:i+2])
                ints = set.intersection(scut,qset)
                if len(ints) > 0:
                    cidf = 0
                    for el in ints:
                        cidf+=idfdict[el]
                    for el in ints:
                        for k in ints:
                            if el != k:
                                if (ltext.index(el) - ltext.index(k))*(query.index(el) - query.index(k)) > 0:
                                    cidf *= 1.1
                                if abs((ltext.index(el) - ltext.index(k)))<=abs((query.index(el) - query.index(k))):
                                    cidf *= 1.05
                    pas+=cidf*(len(ltext) - i)
            fout.write(str(key) + '\t' + str(pas) + '\n')
    return



In [ ]:
for q in tqdm.tqdm(dctqs.keys()):
    getpscore(q)

In [ ]:
outdata = './features/pas3/'
def getpsmallscore(qid):
    if qid+'.txt' in os.listdir(outdata):
        return
    global dctqs
    global id_q
    texts = {}
    idfdict = {}
    for doc in dctqs[qid]:
        try:
            with open(data+doc+'.txt') as fin:
                texts[doc] = word_tokenize(fin.readline())
                for w in texts[doc]:
                    if w not in idfdict.keys():
                        idfdict[w] = 0
                    idfdict[w] += 1
        except:
            continue
    for key in idfdict.keys():
        idfdict[key] = len(texts) / idfdict[key]
    query = word_tokenize(id_q[qid])
    qset = set(query)
    with open(outdata+qid+'.txt','w') as fout:
        for key in texts.keys():
            ltext = texts[key]
            pas = 0
            intl = []
            for i in range(0,len(ltext) - 3,2):
                scut = set(ltext[i:i+3])
                ints = set.intersection(scut,qset)
                intl.append(ints)
                if len(ints) > 0:
                    cidf = 0
                    for el in ints:
                        cidf+=idfdict[el]
                    for el in ints:
                        for k in ints:
                            if el != k:
                                if (ltext.index(el) - ltext.index(k))*(query.index(el) - query.index(k)) > 0:
                                    cidf *= 1.1
                                if abs((ltext.index(el) - ltext.index(k)))<=abs((query.index(el) - query.index(k))):
                                    cidf *= 1.05
                    pas+=cidf*(len(ltext) - 3 - i)
            fout.write(str(key) + '\t' + str(pas) + '\n')
    return

outdata = './features/pas/'
def getpscore(qid):
    if qid+'.txt' in os.listdir(outdata):
        return
    global dctqs
    global id_q
    texts = {}
    idfdict = {}
    for doc in dctqs[qid]:
        try:
            with open(data+doc+'.txt') as fin:
                texts[doc] = word_tokenize(fin.readline())
                for w in texts[doc]:
                    if w not in idfdict.keys():
                        idfdict[w] = 0
                    idfdict[w] += 1
        except:
            continue
    for key in idfdict.keys():
        idfdict[key] = len(texts) / idfdict[key]
    query = word_tokenize(id_q[qid])
    qset = set(query)
    with open(outdata+qid+'.txt','w') as fout:
        for key in texts.keys():
            ltext = texts[key]
            pas = 0
            intl = []
            for i in range(0,len(ltext) - 7,3):
                scut = set(ltext[i:i+7])
                ints = set.intersection(scut,qset)
                intl.append(ints)
                if len(ints) > 0:
                    cidf = 0
                    for el in ints:
                        cidf+=idfdict[el]
                    for el in ints:
                        for k in ints:
                            if el != k:
                                if (ltext.index(el) - ltext.index(k))*(query.index(el) - query.index(k)) > 0:
                                    cidf *= 1.1
                                if abs((ltext.index(el) - ltext.index(k)))<=abs((query.index(el) - query.index(k))):
                                    cidf *= 1.05
                    pas+=cidf*(len(ltext) - 7 - i)
            fout.write(str(key) + '\t' + str(pas) + '\n')
    return


In [ ]:
for q in tqdm.tqdm(dctqs.keys()):
    getpscore(q)

In [ ]:
for q in tqdm.tqdm(dctqs.keys()):
    getpsmallscore(q)

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30
from pyaspeller import YandexSpeller
import pymorphy2
import locale
from pymystem3 import Mystem
from multiprocessing import Pool
from p_tqdm import p_imap
from sklearn.metrics.pairwise import cosine_similarity as cosine
import scipy.spatial.distance as ds
import tqdm
from p_tqdm import p_map
import fasttext.util
import matplotlib
import matplotlib.pyplot as plt
import copy
import os
import xgboost as xgb
from scipy.special import expit
from collections import defaultdict
from sklearn.tree import DecisionTreeRegressor
from multiprocessing import Pool
from scipy.sparse import csr_matrix
import lightgbm as gbm

In [ ]:
speller = YandexSpeller(lang='en', find_repeat_words=False,
                                       ignore_digits=True, max_requests=555)
morph = pymorphy2.MorphAnalyzer()
def stem(line):
    l = line.split('\t')
    text = l[1].strip()
    qid=l[0]
    m = Mystem()
    lemmas = m.lemmatize(text.lower())
    return qid,''.join(lemmas)

In [ ]:
queries = {}
pool = Pool(6)
file = open("normal_queries.txt", 'w', encoding = 'utf-8')
with open("qspell.txt", 'r', encoding="utf-8") as f:
    lines= f.readlines()
    for qid, norm_text in p_map(stem,lines):
        file.write(qid+'\t'+norm_text)
        queries[qid] = norm_text
file.close()

In [ ]:
file =open("normal_queries.txt" ,'r', encoding='utf-8')
queries = {}
for line in file.readlines():
    l=line.split('\t')
    queries[int(l[0])]= l[1].strip()
file.close()

In [ ]:
file =open("qspell.txt" ,'r', encoding='utf-8')
queries_not_normal = {}
for line in file.readlines():
    l=line.split('\t')
    queries_not_normal[int(l[0])]= l[1].strip()
file.close()

In [ ]:
file =open("title.txt" ,'r', encoding='utf-8')
titles_not_normal= {}
for line in file.readlines():
    l=line.split('\t')
    titles_not_normal[int(l[0])]= l[1].strip()
file.close()

In [ ]:
file =open("normal_titles.txt" ,'r', encoding='utf-8')
titles= {}
for line in file.readlines():
    l=line.split('\t')
    titles[int(l[0])]= l[1].strip()
file.close()

In [ ]:
file = open("train.marks.tsv/train.marks.tsv", 'r')
train=[]
train_map={}
queries_doc_map_all={}
marks={}
for l in tqdm.tqdm(file.readlines()):
    splits=l.split('\t')
    if int(splits[2])==0:
        continue
    train.append([int(splits[0]),int(splits[1]), int(splits[2])])
    marks[splits[0]+"|"+splits[1]]=int(splits[2])
    train_map[splits[0]+'|'+splits[1]]=splits[2].strip()
    if int(splits[0]) not in queries_doc_map_all.keys():
        queries_doc_map_all[int(splits[0])]=[int(splits[1])]
    else:
        queries_doc_map_all[int(splits[0])].append(int(splits[1]))
train=np.array(train)
file.close()
test_map={}
file = open("sample.csv/sample.csv", 'r')
file.readline()
test=[]
for l in tqdm.tqdm(file.readlines()):
    splits=l.split(',')
    test.append([int(splits[0]),int(splits[1]), -1])
    test_map[splits[0]+'|'+splits[1].strip()]="-1"
    if int(splits[0]) not in queries_doc_map_all.keys():
        queries_doc_map_all[int(splits[0])]=[int(splits[1])]
    else:
        queries_doc_map_all[int(splits[0])].append(int(splits[1]))
file.close()

In [ ]:
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi, BM25Plus, BM25L
from stop_words import get_stop_words
stop = get_stop_words('ru')+get_stop_words('en')+['aren', 'can', 
    'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll',
        'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn']

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def get_ngrams(text, n=1, word=True):
    if word:
        n_grams = ngrams(word_tokenize(text), n)
        return [ ' '.join(grams) for grams in n_grams]
    else:
        n_grams = ngrams(text, n)
        return [ ''.join(grams) for grams in n_grams]

In [ ]:
def generator_titles(doc_ids, bm=False, n=1, word=True, doc2vec=False):
    for i in doc_ids:
        try:
            if bm:
                yield get_ngrams(titles[i], n=n, word=word)
            elif doc2vec:
                yield i,titles[i]
            else:
                yield titles[i]
        except:
            if doc2vec:
                yield i,"0"
            else:
                yield "null"

In [ ]:
def generator_titles_tr(doc_ids, bm=False, n=1, word=True, doc2vec=False):
    for i in doc_ids:
        try:
            if bm:
                yield get_ngrams(tr_titles[i], n=n, word=word)
            elif doc2vec:
                yield i,tr_titles[i]
            else:
                yield tr_titles[i]
        except:
            if doc2vec:
                yield i,"0"
            else:
                yield "null"

In [ ]:
def save_features(filename, features):
    with open(filename,'w', encoding='utf-8') as f:
        for i in features.keys():
            f.write(str(i)+'\t')
            for j in features[i]:
                if not np.isfinite(j) or j<0:
                    j=0
                f.write(str(j)+'\t')
            f.write('\n')

In [ ]:
def generator_doc(qid):
    doc={}
    f=open("docs/{}.txt".format(qid), 'r', encoding='utf-8')
    for line in f.readlines():
        l=line.split('\t')
        doc[int(l[0])]=l[1].strip()
    f.close()
    for i in queries_doc_map_all[qid]:
        if i in doc.keys():
            yield doc[i]
        else:
            yield "not found"
    del doc

In [ ]:
features_tfidf={}
for i in tqdm.tqdm(range(0,len(queries_doc_map_all))):
    vectorizer = TfidfVectorizer(ngram_range=(1,1), stop_words=None)
    X=vectorizer.fit_transform(generator_titles(queries_doc_map_all[i]))
    q=vectorizer.transform([queries[i]])
    cos9=cosine(X,q).ravel()
    cos1=(((X.dot(q.T)).toarray()).T).reshape(X.shape[0])
    vectorizer = TfidfVectorizer(ngram_range=(2,2), stop_words=stop)
    X=vectorizer.fit_transform(generator_titles(queries_doc_map_all[i]))
    q=vectorizer.transform([queries[i]])
    cos2=(((X.dot(q.T)).toarray()).T).reshape(X.shape[0])
    cos10=cosine(X,q).ravel()
    vectorizer = TfidfVectorizer(ngram_range=(1,3), stop_words=stop)
    X=vectorizer.fit_transform(generator_titles(queries_doc_map_all[i]))
    q=vectorizer.transform([queries[i]])
    cos3=(((X.dot(q.T)).toarray()).T).reshape(X.shape[0])
    cos11=cosine(X,q).ravel()
    vectorizer = TfidfVectorizer(ngram_range=(4,13), stop_words=stop, analyzer='char_wb')
    X=vectorizer.fit_transform(generator_titles(queries_doc_map_all[i]))
    q=vectorizer.transform([queries[i]])
    cos4=(((X.dot(q.T)).toarray()).T).reshape(X.shape[0])
    cos12=cosine(X,q).ravel()
    
    
    vectorizer = TfidfVectorizer(ngram_range=(1,1),use_idf=False,stop_words=None)
    X=vectorizer.fit_transform(generator_titles(queries_doc_map_all[i]))
    q=vectorizer.transform([queries[i]])
    cos5=(((X.dot(q.T)).toarray()).T).reshape(X.shape[0])
    cos13=cosine(X,q).ravel()
    vectorizer = TfidfVectorizer(ngram_range=(2,2), stop_words=stop,use_idf=False)
    X=vectorizer.fit_transform(generator_titles(queries_doc_map_all[i]))
    q=vectorizer.transform([queries[i]])
    cos6=(((X.dot(q.T)).toarray()).T).reshape(X.shape[0])
    cos14=cosine(X,q).ravel()
    vectorizer = TfidfVectorizer(ngram_range=(1,3), stop_words=stop, use_idf=False)
    X=vectorizer.fit_transform(generator_titles(queries_doc_map_all[i]))
    q=vectorizer.transform([queries[i]])
    cos7=(((X.dot(q.T)).toarray()).T).reshape(X.shape[0])
    cos15=cosine(X,q).ravel()
    vectorizer = TfidfVectorizer(ngram_range=(4,13), stop_words=stop, analyzer='char_wb', use_idf=False)
    X=vectorizer.fit_transform(generator_titles(queries_doc_map_all[i]))
    q=vectorizer.transform([queries[i]])
    cos8=(((X.dot(q.T)).toarray()).T).reshape(X.shape[0])
    cos16=cosine(X,q).ravel()
      
    
    sents=list(generator_titles(queries_doc_map_all[i], bm=True, n=1))
    bm25 = BM25Plus(sents)
    res1=bm25.get_scores(get_ngrams(queries[i], n=1, word=True))
    sents=list(generator_titles(queries_doc_map_all[i], bm=True, n=2))
    bm25 = BM25Plus(sents)
    res2=bm25.get_scores(get_ngrams(queries[i], n=2, word=True))
    sents=list(generator_titles(queries_doc_map_all[i], bm=True, n=3, word=False))
    bm25 = BM25Plus(sents)
    res3=bm25.get_scores(get_ngrams(queries[i], n=3, word=False))
    sents=list(generator_titles(queries_doc_map_all[i], bm=True, n=5, word=False))
    bm25 = BM25Plus(sents)
    res4=bm25.get_scores(get_ngrams(queries[i], n=5, word=False))
    
    
    sents=list(generator_titles(queries_doc_map_all[i], bm=True, n=1))
    bm25 = BM25L(sents)
    res5=bm25.get_scores(get_ngrams(queries[i], n=1, word=True))
    sents=list(generator_titles(queries_doc_map_all[i], bm=True, n=2))
    bm2 = BM25L(sents)
    res6=bm25.get_scores(get_ngrams(queries[i], n=2, word=True))
    sents=list(generator_titles(queries_doc_map_all[i], bm=True, n=3, word=False))
    bm25 = BM25L(sents)
    res7=bm25.get_scores(get_ngrams(queries[i], n=3, word=False))
    sents=list(generator_titles(queries_doc_map_all[i], bm=True, n=5, word=False))
    bm25 = BM25L(sents)
    res8=bm25.get_scores(get_ngrams(queries[i], n=5, word=False))
    
    sents=list(generator_titles(queries_doc_map_all[i], bm=True, n=1))
    bm25 = BM25Okapi(sents)
    res9=bm25.get_scores(get_ngrams(queries[i], n=1, word=True))
    sents=list(generator_titles(queries_doc_map_all[i], bm=True, n=2))
    bm2 = BM25Okapi(sents)
    res10=bm25.get_scores(get_ngrams(queries[i], n=2, word=True))
    sents=list(generator_titles(queries_doc_map_all[i], bm=True, n=3, word=False))
    bm25 = BM25Okapi(sents)
    res11=bm25.get_scores(get_ngrams(queries[i], n=3, word=False))
    sents=list(generator_titles(queries_doc_map_all[i], bm=True, n=5, word=False))
    bm25 = BM25Okapi(sents)
    res12=bm25.get_scores(get_ngrams(queries[i], n=5, word=False))
    
    for k,j in enumerate(queries_doc_map_all[i]):
        features_tfidf[str(i)+"|"+str(j)]=[cos1[k],cos2[k], cos3[k],cos4[k],cos5[k],cos6[k], cos7[k],cos8[k],
                       cos9[k],cos10[k], cos11[k],cos12[k],cos13[k],cos14[k], cos15[k],cos16[k],res1[k], res2[k], res3[k], res4[k],
                                              res5[k], res6[k], res7[k], res8[k], res9[k], res10[k], res11[k], res12[k]]

In [ ]:
def save_features(filename, features):
    with open(filename,'w', encoding='utf-8') as f:
        for i in features.keys():
            f.write(str(i)+'\t')
            for j in features[i]:
                if not np.isfinite(j) or j<0:
                    j=0
                f.write(str(j)+'\t')
            f.write('\n')

In [ ]:
save_features('features/tfidf.txt', features_tfidf)


In [ ]:
import wget
import gensim
import zipfile

import fasttext.util
model1=gensim.models.KeyedVectors.load("/Users/lmac/Downloads/181/model.model")
model2=gensim.models.KeyedVectors.load("/Users/lmac/Downloads/187/model.model")
ft_en = fasttext.load_model('/Users/lmac/Downloads/cc.en.300.bin')
ft_ru = fasttext.load_model('/Users/lmac/Downloads/cc.ru.300.bin')
ft_new= fasttext.load_model('/Users/lmac/Downloads/ft_native_300_ru_wiki_lenta_lemmatize.bin')


In [ ]:
def sim(qid):
    res=[]
    q_vector1=np.zeros(300)
    q_vector2=np.zeros(300)
    q_vector3=np.zeros(300)
    q_vector4=np.zeros(300)
    q_vector8=np.zeros(300)
    q_vector5=ft_ru.get_sentence_vector(queries[qid])
    q_vector6=ft_en.get_sentence_vector(queries[qid])
    q_vector7=ft_new.get_sentence_vector(queries[qid])
    for w in queries[qid].split():
        q_vector1+=model1.get_vector(w)
        q_vector2+=model2.get_vector(w)
        q_vector3+=ft_ru.get_word_vector(w)
        q_vector4+=ft_en.get_word_vector(w)
        q_vector8+=ft_new.get_word_vector(w)
    for title in list(generator_titles(queries_doc_map_all[qid])):
        d_vector1=np.zeros(300)
        d_vector2=np.zeros(300)
        d_vector3=np.zeros(300)
        d_vector4=np.zeros(300)
        d_vector8=np.zeros(300)
        d_vector5=ft_ru.get_sentence_vector(title)
        d_vector6=ft_en.get_sentence_vector(title)
        d_vector7=ft_new.get_sentence_vector(title)
        for w in title.split():
            d_vector1+=model1.get_vector(w)
            d_vector2+=model2.get_vector(w)
            d_vector3+=ft_ru.get_word_vector(w)
            d_vector4+=ft_en.get_word_vector(w)
            d_vector8+=ft_new.get_word_vector(w)
        s=[model1.similarity(queries[qid], title),
        model2.similarity(queries[qid], title),
        ds.cosine(q_vector1,d_vector1),
        ds.cosine(q_vector2,d_vector2),
        ds.cosine(q_vector3,d_vector3),
        ds.cosine(q_vector4,d_vector4),
        ds.cosine(q_vector5,d_vector5),
        ds.cosine(q_vector6,d_vector6),
        ds.cosine(q_vector7,d_vector7),
        ds.cosine(q_vector8,d_vector8),
        np.dot(q_vector1,d_vector1),
        np.dot(q_vector2,d_vector2),
        np.dot(q_vector3,d_vector3),
        np.dot(q_vector4,d_vector4),
        np.dot(q_vector5,d_vector5),
        np.dot(q_vector6,d_vector6),
        np.dot(q_vector7,d_vector7),
        np.dot(q_vector8,d_vector8)]
        res.append(s)
    return res

In [ ]:
features_fasttext={}
for qid in tqdm.tqdm(range(6311)):
    for i,j in zip(queries_doc_map_all[qid], sim(qid)):
        features_fasttext[str(qid)+"|"+str(i)]=j

In [ ]:
save_features("features/fasttext.txt", features_fasttext)

In [ ]:
import pickle
pickle_obj = open("features_fasttext.pickle","wb")
pickle.dump(features_fasttext, pickle_obj)
pickle_obj.close()

In [ ]:
file =open("normal_titles.txt" ,'r', encoding='utf-8')
titles= []
id_titles=[]
for line in file.readlines():
    l=line.split('\t')
    titles.append(l[1].strip())
    id_titles.append(int(l[0]))
file.close()

In [ ]:
from sentence_transformers import SentenceTransformer, models
model1 = SentenceTransformer('distiluse-base-multilingual-cased')


In [ ]:
vectors_from_model1=model1.encode(titles, show_progress_bar=True, batch_size=128)
titles_={}
for i,j in zip(id_titles,vectors_from_model1):
    titles_[i]=j

In [ ]:
with open("features/bert.txt", 'w', encoding='utf-8') as f:
    for qid in tqdm.tqdm(range(len(queries))):
        q_emb=model1.encode([queries[qid]])[0]
        for i in queries_doc_map_all[qid]:
            if i in titles_.keys():
                f.write(str(qid)+"|"+str(i)+'\t'+str(ds.cosine(q_emb, titles_[i]))+'\n')


In [ ]:
import torch
from transformers import *
tokenizer1 = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-conversational", use_fast=True)
model1 = AutoModelWithLMHead.from_pretrained("DeepPavlov/rubert-base-cased-conversational")

tokenizer2 = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased", use_fast=True)
model2 = AutoModelWithLMHead.from_pretrained("DeepPavlov/rubert-base-cased")

tokenizer3 = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence", use_fast=True)
model3 = AutoModelWithLMHead.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [ ]:
def sim_bert(qid):
    res=[]
    device = 'cpu'
    q_token = tokenizer1.encode_plus(queries[qid], return_tensors="pt").to(device)
    q_emb1 = model1(**q_token)[0].to(torch.device('cpu')).detach().numpy()
    q_token = tokenizer2.encode_plus(queries[qid], return_tensors="pt").to(device)
    q_emb2 = model2(**q_token)[0].to(torch.device('cpu')).detach().numpy()
    q_token = tokenizer3.encode_plus(queries[qid], return_tensors="pt").to(device)
    q_emb3 = model3(**q_token)[0].to(torch.device('cpu')).detach().numpy()
    for title in generator_titles(queries_doc_map_all[qid]):
        d_token = tokenizer1.encode_plus(title, return_tensors="pt").to(device)
        d_emb1 = model1(**d_token)[0].to(torch.device('cpu')).detach().numpy()
        d_token = tokenizer2.encode_plus(title, return_tensors="pt").to(device)
        d_emb2 = model2(**d_token)[0].to(torch.device('cpu')).detach().numpy()
        d_token = tokenizer3.encode_plus(title, return_tensors="pt").to(device)
        d_emb3 = model3(**d_token)[0].to(torch.device('cpu')).detach().numpy()
        res.append([cosine(q_emb1[0], d_emb1[0]).mean(),cosine(q_emb1.sum(1), d_emb1.sum(1)).ravel()[0],
                   cosine(q_emb2[0], d_emb2[0]).mean(),cosine(q_emb2.sum(1), d_emb2.sum(1)).ravel()[0],
                   cosine(q_emb3[0], d_emb3[0]).mean(),cosine(q_emb3.sum(1), d_emb3.sum(1)).ravel()[0]])
        
    return res

In [ ]:
def sim_bert(qid):
    global queries
    global queries_doc_map_all
    global generator_titles
    
    res=[]
    device = 'cpu'
    q_token = tokenizer1.encode_plus(queries[qid], return_tensors="pt").to(device)
    q_emb1 = model1(**q_token)[0].to(torch.device('cpu')).detach().numpy()
    q_token = tokenizer2.encode_plus(queries[qid], return_tensors="pt").to(device)
    q_emb2 = model2(**q_token)[0].to(torch.device('cpu')).detach().numpy()
    q_token = tokenizer3.encode_plus(queries[qid], return_tensors="pt").to(device)
    q_emb3 = model3(**q_token)[0].to(torch.device('cpu')).detach().numpy()
    for title in generator_titles(queries_doc_map_all[qid]):
        d_token = tokenizer1.encode_plus(title, return_tensors="pt").to(device)
        d_emb1 = model1(**d_token)[0].to(torch.device('cpu')).detach().numpy()
        d_token = tokenizer2.encode_plus(title, return_tensors="pt").to(device)
        d_emb2 = model2(**d_token)[0].to(torch.device('cpu')).detach().numpy()
        d_token = tokenizer3.encode_plus(title, return_tensors="pt").to(device)
        d_emb3 = model3(**d_token)[0].to(torch.device('cpu')).detach().numpy()
        res.append([cosine(q_emb1[0], d_emb1[0]).mean(),cosine(q_emb1.sum(1), d_emb1.sum(1)).ravel()[0],
                   cosine(q_emb2[0], d_emb2[0]).mean(),cosine(q_emb2.sum(1), d_emb2.sum(1)).ravel()[0],
                   cosine(q_emb3[0], d_emb3[0]).mean(),cosine(q_emb3.sum(1), d_emb3.sum(1)).ravel()[0]])
    with open('./features/deeppavlov/{}.txt'.format(qid),'w') as fout:
        for i,j in zip(queries_doc_map_all[qid], res):
            fout.write(str(qid)+"|"+str(i)+'\t')
            for k in j:
                fout.write(str(k)+'\t')
            fout.write('\n')
        
    return res

In [ ]:
file= open("features/transforms.txt", 'a', encoding='utf-8')
for qid in tqdm.tqdm(range(0,6311)):
    for i,j in zip(queries_doc_map_all[qid], sim_bert(qid)):
        file.write(str(qid)+"|"+str(i)+'\t')
        for k in j:
            file.write(str(k)+'\t')
        file.write('\n')
file.close()

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

#embed1 = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")
#embed2 = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
module3 = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')


In [ ]:
def sim_USE(qid):
    questions = [queries_not_normal[qid]]
    question_embeddings = module3.signatures['question_encoder'](
            tf.constant(questions))
    responses = list(generator_titles(queries_doc_map_all[qid]))
    response_contexts = responses
    response_embeddings = module3.signatures['response_encoder'](
        input=tf.constant(responses),
        context=tf.constant(response_contexts))
    sim1 = np.inner(question_embeddings['outputs'], response_embeddings['outputs']).ravel()
    #sim2 = cosine(embed1(queries_not_normal[qid]), embed1(list(generator_titles_not_normal(queries_doc_map_all[qid])))).ravel()
    #sim3 = cosine(embed2(queries_not_normal[qid]), embed2(list(generator_titles_not_normal(queries_doc_map_all[qid])))).ravel()
    return np.hstack((sim1.reshape(-1,1))

In [ ]:
import tqdm

features_USE_not_normal={}
with open("features/USE_not_normal.txt", 'a+') as f:
    for qid in tqdm.tqdm(range(1203, 6311)):
        for i,j in zip(queries_doc_map_all[qid], sim_USE(qid)):
            features_USE_not_normal[str(qid)+"|"+str(i)]=j
            f.write(str(qid)+"|"+str(i)+'\t'+str(j)+'\n')

In [ ]:
sim_USE(207)

In [ ]:
import numpy as np
import tqdm
import operator
import lightgbm as lg
import os

In [ ]:
#data 
queries = {}
with open('./normal_queries.txt') as fin:
    lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if line[0] == '':
            line.pop(0)
        queries[line[0]] = line[1]

titles= {}
with open("./normal_titles.txt" ,'r', encoding='utf-8') as fin:
    for line in fin.readlines():
        line=line.split('\t')
        titles[line[0]]= line[1][:-1]

train=[]
with open("./train.marks.tsv/train.marks.tsv", 'r') as fin:
    for line in fin.readlines():
        line=line[:-1].split('\t')
        train.append([line[0],line[1], line[2]])

                
test=[]
with open("./sample.csv/sample.csv", 'r') as fin:
    fin.readline()
    for line in fin.readlines():
        line=line[:-1].split(',')
        test.append([line[0],line[1],-1])

In [ ]:
hostdict = {}
with open('./url.data/url.data') as fin:
    lines = fin.readlines()
for line in lines:
    line = line[:-1].split('\t')
    if line[1][:7] == 'http://':
        line[1] = line[1][7:]
    if line[1][:4] == 'www.':
        line[1] = line[1][4:]
    hostdict[line[0]] = line[1].split('/')[0]

In [ ]:
similars = {}
with open('./features/prevsimdoc.txt', 'r') as fin:
    lines = fin.readlines()
    for line in lines:
        spl = line[:-2].split('\t')
        if len(spl) == 1:
            continue
        tmp = spl[1].split(' ')
        for i in range(13):
            tmp.pop()
        similars[spl[0]] = tmp

In [ ]:
file =open("./features/dvs.txt" ,'r', encoding='utf-8')
dvs= {}
for line in file.readlines():
    l=line.split('\t')
    dvs[(l[0], l[1])]= np.array(l[2][:-1], dtype = np.float64)
file.close()

In [ ]:
#pas.py
pas = {}
pasfiles = os.listdir('./features/pas')
for file in pasfiles:
    with open('./features/pas/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if len(line) > 1:
            pas[(file[:-4], line[0])] = np.log(1 + np.float64(line[1]))


In [ ]:
muse = {}
musefiles = os.listdir('./features/muse')
for file in musefiles:
    with open('./features/muse/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if len(line) > 1:
            muse[(line[0], line[1])] = np.float64(line[2])

In [ ]:
past = {}
pastfiles = os.listdir('./features/pasT')
for file in pastfiles:
    with open('./features/pasT/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if len(line) > 1:
            past[(file[:-4], line[0])] = np.log(1+ np.float64(line[1]))
            
pas3 = {}
pas3files = os.listdir('./features/pas3')
for file in pas3files:
    with open('./features/pas3/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if len(line) > 1:
            pas3[(file[:-4], line[0])] = np.log(1+ np.float64(line[1]))


In [ ]:
luse = {}
with open('./features/luse.txt') as fin:
    lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        luse[(line[0], line[1])] = np.float64(line[2])


In [ ]:
trft = {}
trftfiles = os.listdir('./features/trft')
for file in pastfiles:
    with open('./features/trft/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if len(line) > 1:
            trft[(file[:-4], line[0])] = np.float64(line[1])

In [ ]:
file =open("./features/DBN/part-r-00000" ,'r', encoding='utf-8')
dbndoc = {}
for line in file.readlines():
    l=line[:-1].split('\t')
    k = l[0]
    l.pop(0)
    dbndoc[k]= np.array(l, dtype = np.float64)
file.close()

In [ ]:
hostfiles = os.listdir('./features/DDF')
hf = {}
qhf = {}
c = 0
for file in hostfiles:
    with open('./features/DDF/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        line.pop()
        if file[0] == 'Q':
            q = line.pop(0)
            h = line.pop(0)
            qhf[(q,h)] = np.array(line, dtype = np.float64)
            c+=1
        else:
            h = line.pop(0)
            hf[h] = np.array(line, dtype = np.float64)

In [ ]:
urlfiles = os.listdir('./features/UDDF')
uf = {}
quf = {}
for file in urlfiles:
    with open('./features/UDDF/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        line.pop()
        if file[1] == 'Q':
            q = line.pop(0)
            u = line.pop(0)
            quf[(q,u)] = np.array(line, dtype = np.float64)
        else:
            u = line.pop(0)
            uf[u] = np.array(line, dtype = np.float64)

In [ ]:
file =open("./features/QF/part-r-00000" ,'r', encoding='utf-8')
qf = {}
for line in file.readlines():
    l=line[:-1].split('\t')
    k = l[0]
    l.pop(0)
    qf[k]= np.array(l, dtype = np.float64)
file.close()

In [ ]:
quse = {}
with open('./features/quse.txt') as fin:
    lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        quse[(line[0], line[1])] = np.float64(line[2])

In [ ]:
def getdata(cat):
    trainx = []
    trainy = []
    qid_train = []
    index = []
    for entry in tqdm.tqdm(cat):
        try:
            try:
                tempx = dvs[(entry[0],entry[1])]
            except:
                tempx = np.array([0.0])
            try:
                tempx = np.hstack((tempx,trft[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,muse[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,past[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            trainx.append(tempx)
            trainy.append(float(entry[2]))
            qid_train.append(int(entry[0]))
            index.append((entry[0], entry[1]))
        except:
            continue
    Xtrain = np.row_stack([row for row in trainx])
    Ytrain = np.row_stack([row for row in trainy])
    return Xtrain, Ytrain, qid_train, index

In [ ]:
Xtrain, Ytrain, qid_train, tindex = getdata(train)

In [ ]:
def read_features(filename):
    fet={}
    with open(filename,'r', encoding='utf-8') as f:
        for line in f.readlines():
            l=line.strip().split('\t')
            x=[]
            for i in range(1, len(l)):
                x.append(float(l[i].strip()))
            fet[l[0]]=x
    return fet

In [ ]:
features_tfidf=read_features("features/tfidf.txt")
features_fasttext=read_features("features/fasttext.txt")
features_bert=read_features("features/bert.txt")
features_transforms = read_features("features/transforms.txt")


In [ ]:
map_hosts={}
set_hosts =set()
with open("url.data/url.data", 'r', encoding='utf-8') as f:
    for line in f.readlines():
        l=line.split('\t')
        host=l[1].split("/")[0]
        host = host[4:] if host.startswith("www.") else host
        doc_id = int(l[0])
        map_hosts[host]=doc_id
        set_hosts.add(host)
hosts={}
with open("url.data/url.data", 'r', encoding='utf-8') as f:
    for line in f.readlines():
        l=line.split('\t')
        host=l[1].split("/")[0]
        host = host[4:] if host.startswith("www.") else host
        host_id = map_hosts[host]    
        doc_id = int(l[0])
        if host_id in hosts.keys():
            hosts[host_id].append(doc_id)
        else:
            hosts[host_id] = []
            hosts[host_id].append(doc_id)

In [ ]:
docs_sDBN = {}
file = open("features/new_sDBN/DOCS/part-r-00000",'r')
for l in file.readlines():
    d=[]
    splits=l.split('\t')
    d.append(float(splits[1]))
    d.append(float(splits[2]))
    d.append(float(splits[3]))
    docs_sDBN[int(splits[0])]=np.array(d)
file.close()
qdocs_sDBN = {}
file = open("features/new_sDBN/QDOCS/part-r-00000",'r')
for l in file.readlines():
    d=[]
    splits=l.split('\t')
    d.append(float(splits[2]))
    d.append(float(splits[3]))
    d.append(float(splits[4]))
    qdocs_sDBN[splits[0]+"|"+splits[1]]=np.array(d)
file.close()
host_sDBN = {}
file = open("features/new_sDBN/HOST/part-r-00000",'r')
for l in file.readlines():
    d=[]
    splits=l.split('\t')
    hid= int(splits[0])
    d.append(float(splits[1]))
    d.append(float(splits[2]))
    d.append(float(splits[3]))
    if hid in hosts.keys():
        for host in hosts[hid]:
            host_sDBN[host] =np.array(d)
file.close()
qhost_sDBN = {}
file = open("features/new_sDBN/QHOST/part-r-00000",'r')
for l in file.readlines():
    d=[]
    splits=l.split('\t')
    hid= int(splits[1])
    d.append(float(splits[2]))
    d.append(float(splits[3]))
    d.append(float(splits[4]))
    if hid in hosts.keys():
        for host in hosts[hid]:
            if host in queries_doc_map_all[int(splits[0])]:
                qhost_sDBN[splits[0]+"|"+str(host)]=np.array(d)
file.close()

In [ ]:
def read_data(filename, Q=False):
    Docs = {}
    if Q:
        start=2
        end=13
    else:
        start=1
        end=12
    for i in range(11):
        with open("features/new_PBM/"+filename+"/part-r-000"+str(i).zfill(2), 'r', encoding='utf-8') as f:
            for line in f.readlines():
                data={}
                l = line.strip().split('\t')
                for k in range(start,end):
                    splt = l[k].strip().split(":")
                    data[splt[0]]= float(splt[1])
                for k in range(end,end+3):
                    splt = l[k].split(":")
                    pos = []
                    for m,j in enumerate(splt[1].strip().split(" ")):
                        if m == 10:
                            data[splt[0]+'_last']=int(j)
                            break
                        pos.append(int(j))
                    data[splt[0]]=np.array(pos)
                if Q:
                    Docs[l[0]+'|'+l[1]]=data
                else:
                    Docs[int(l[0])]=data
    return Docs

In [ ]:
def make_features(Docs, Q=False,H=False):
    features = {}
    #alpha = np.arange(0.05, 0.45, 0.01)
    alpha = np.array([0.41,0.16,0.105,0.08,0.06,0.05,0.35,0.3,0.25,0.2])
    for i in Docs.keys():
        #if Docs[i]['clicks']<=1:
        #    continue
        CTR = Docs[i]['clicks']/Docs[i]['shows']
        AvgTime = Docs[i]['time']/Docs[i]['clicks'] if Docs[i]['clicks']!=0 else 0
        FCTR = Docs[i]['first_click']/Docs[i]['shows']
        LCTR = Docs[i]['last_click']/Docs[i]['shows']
        AVGPos = np.mean(Docs[i]['shows_pos'])
        AVGClick = np.mean(Docs[i]['click_pos'])
        AVGPosClick = np.mean(Docs[i]['show_pos_if_click'])
        Shows10 = Docs[i]['shows_pos_last']/Docs[i]['shows']
        Clicks10 = Docs[i]['click_pos_last']/Docs[i]['clicks'] if Docs[i]['clicks']!=0 else 0
        LastProb = Docs[i]['last_click']/Docs[i]['clicks'] if Docs[i]['clicks']!=0 else 0
        NotFirst = np.sum(Docs[i]['click_pos'][1:])/Docs[i]['shows']
        CTRPos = 1.*Docs[i]['show_pos_if_click']/Docs[i]['shows_pos'] #массив
        Prob_Click_pos = Docs[i]['click_pos']/Docs[i]['clicks'] #массив
        Prob_Click_showpos= Docs[i]['show_pos_if_click']/Docs[i]['clicks'] #массив
        PBM = Docs[i]['clicks']/np.sum(Docs[i]['shows_pos']*alpha)
        CM_shows =  Docs[i]['cm_clicks']/Docs[i]['cm_shows']  if Docs[i]['cm_shows']!=0 else 0
        UpProb = Docs[i]['up_click']/Docs[i]['shows']
        DownProb = Docs[i]['down_click']/Docs[i]['shows']
        AVGbeforeShowDocs=Docs[i]['before_click']/(Docs[i]['clicks']-Docs[i]['first_click']) if (Docs[i]['clicks']-Docs[i]['first_click'])!=0 else 0
        AVGafterShowDocs=Docs[i]['after_click']/(Docs[i]['clicks']-Docs[i]['last_click'])if (Docs[i]['clicks']-Docs[i]['last_click'])!=0 else 0
        CTR5=np.sum(Docs[i]['show_pos_if_click'][0:5])/np.sum(Docs[i]['shows_pos'][0:5]) if np.sum(Docs[i]['shows_pos'][0:5])!=0 else 0
        logShows =  np.log(1+Docs[i]['shows'])
        logClicks =  np.log(1+Docs[i]['clicks'])
        data = [CTR, AvgTime, FCTR, LCTR, AVGPos, AVGClick, AVGPosClick, Shows10, Clicks10,LastProb,AVGbeforeShowDocs,AVGafterShowDocs,CTR5,
               NotFirst,PBM, CM_shows, logClicks, logShows,UpProb,DownProb]
        for k in CTRPos:
            if np.isnan(k):
                k=0
            data.append(k)
        for k in Prob_Click_pos:
            if np.isnan(k):
                k=0
            data.append(k)
        for k in Prob_Click_showpos:
            if np.isnan(k):
                k=0
            data.append(k)
            
            
        if Q and H:
            s=i.split('|')
            if int(s[1]) in hosts.keys():
                for host in hosts[int(s[1])]:
                    if host in queries_doc_map_all[int(s[0])]:
                        features[s[0]+"|"+ str(host)] = data
        
        if H and ~Q:
            if i in hosts.keys():
                for host in hosts[i]:
                    features[host] = data 
        if ~H:
            features[i] = data
    return features

In [ ]:
def read_and_make(filename, Q, H):
    Docs=read_data(filename, Q)
    return make_features(Docs,Q=Q,H=H)

In [ ]:
features_docs =read_and_make("DOCS", Q=False, H=False)
features_hosts =read_and_make("HOST", Q=False, H=True)
features_qdocs=read_and_make("QDOCS", Q=True, H=False)
features_qhosts =read_and_make("QHOST", Q=True, H=True)

In [ ]:
Docs = {}
with open("features/QUERY/part-r-00000", 'r', encoding='utf-8') as f:
    for line in f.readlines():
        data={}
        l = line.strip().split('\t')
        for k in range(1,len(l)):
            splt = l[k].strip().split(":")
            data[splt[0]]= float(splt[1])
        Docs[int(l[0])]=data

In [ ]:
features_queries = {}
for i in Docs.keys():
    AVGDocs = Docs[i]['shows_docs']/Docs[i]['count_query']
    AVGClicks = Docs[i]['clicks_docs']/Docs[i]['count_query']
    AVGtime = Docs[i]['time']/Docs[i]['count_query'] if Docs[i]['time']<0 else 60*30
    AVG_pos = Docs[i]['avg_pos_click']/Docs[i]['count_query']
    AVG_pos_first = Docs[i]['first_click']/Docs[i]['count_query']
    Noclick = Docs[i]['shows_noclick']
    data=[AVGDocs,AVGClicks,AVGtime,AVG_pos,AVG_pos_first,Noclick]
    features_queries[i] = data

In [ ]:
def save_features(filename, features):
    with open(filename,'w', encoding='utf-8') as f:
        for i in features.keys():
            f.write(str(i)+'\t')
            for j in features[i]:
                if not np.isfinite(j) or j<0:
                    j=0
                f.write(str(j)+'\t')
            f.write('\n')

In [ ]:
save_features("features/click_docs.txt", features_docs)
save_features("features/click_qdocs.txt", features_qdocs)
save_features("features/click_hosts.txt", features_hosts)
save_features("features/click_qhosts.txt", features_qhosts)
save_features("features/click_q.txt", features_queries)
save_features("features/sDBN_docs.txt", docs_sDBN)
save_features("features/sDBN_qdocs.txt", qdocs_sDBN)
save_features("features/sDBN_hosts.txt", host_sDBN)
save_features("features/sDBN_qhosts.txt", qhost_sDBN)

In [ ]:
def read_features(filename):
    fet={}
    with open(filename,'r', encoding='utf-8') as f:
        for line in f.readlines():
            l=line.strip().split('\t')
            x=[]
            for i in range(1, len(l)):
                x.append(float(l[i].strip()))
            fet[l[0]]=x
    return fet

In [ ]:
features_docs=read_features("features/click_docs.txt") 
features_qdocs=read_features("features/click_qdocs.txt")
features_hosts=read_features("features/click_hosts.txt")
features_qhosts=read_features("features/click_qhosts.txt")
features_queries=read_features("features/click_q.txt")
docs_sDBN=read_features("features/sDBN_docs.txt")
qdocs_sDBN=read_features("features/sDBN_qdocs.txt")
host_sDBN=read_features("features/sDBN_hosts.txt")
qhost_sDBN=read_features("features/sDBN_qhosts.txt")

In [ ]:
def all_features(t):
    features = {}
    for i in t:
        x1=features_tfidf[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in features_tfidf.keys() else np.zeros(len(features_tfidf['0|340485']))
        x2=features_fasttext[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in features_fasttext.keys() else np.zeros(len(features_fasttext['0|6760']))
        
        x4=features_hosts[str(i[1])] if str(i[1]) in features_hosts.keys() else np.zeros(len(features_hosts['10059']))
        x5=features_docs[str(i[1])] if str(i[1]) in features_docs.keys() else np.zeros(len((features_docs['100059'])))
        x6=qdocs_sDBN[str(i[0])+"|"+str(i[1])]  if str(i[0])+"|"+str(i[1]) in qdocs_sDBN.keys() else np.zeros(3)
        x7=qhost_sDBN[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in qhost_sDBN.keys() else np.zeros(3)
        x8=docs_sDBN[str(i[1])] if str(i[1]) in docs_sDBN.keys() else np.zeros(3)
        x9=host_sDBN[str(i[1])] if str(i[1]) in host_sDBN.keys() else np.zeros(3)
        x10=features_qhosts[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in features_qhosts.keys() else np.zeros(len((features_docs['100059'])))
        x11=features_qdocs[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in features_qdocs.keys() else np.zeros(len((features_docs['100059'])))
        x12=features_queries[str(i[0])] if str(i[0]) in features_queries.keys() else np.zeros(6)        
        
        
        x14=features_bert[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in features_bert.keys() else np.zeros(len(features_bert['0|340485']))
        x15=features_transforms[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in features_transforms.keys() else np.zeros(len(features_transforms['0|340485']))
        try:
            x21 = dvs[(str(i[0]), str(i[1]))]
        except:
            x21 = np.array([0.0])
        try:    
            x22 = trft[(str(i[0]), str(i[1]))]
        except:
            x22 = np.array([0.0])
        try:    
            x23 = muse[(str(i[0]), str(i[1]))]
        except:
            x23 = np.array([0.0])
        try:
            x24 = past[(str(i[0]), str(i[1]))]
        except:
            x24 = np.array([0.0])
        try:
            x30 = texttf[(str(i[0]), str(i[1]))]
        except:
            x30 = np.array([0.0])
        try:
            x31 = newbm[(str(i[0]), str(i[1]))]
        except:
            x31 = np.array([0.0])           
        try:
            x25 = hf[hostdict[str(i[1])]]
        except:

            [x25] = [[0.0] * 13]
        try:
            x26 = qhf[(str(i[0]),hostdict[str(i[1])])]
        except:

            [x26] = [[0.0] * 43]

        try:
            x27 = uf[str(i[1])]
        except:

            [x27] = [[0.0] * 13]

        try:
            x28 = quf[(str(i[0]), str(i[1]))]
        except:

            [x28] = [[0.0] * 43]
        try:
            x29 = dbndoc[str(i[1])]
        except:

            [x29] = [[0.0] * 3]            
        
        features[str(i[0])+"|"+str(i[1])+"|"+str(i[2])]=np.hstack((x1,x2,x14,x15, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30, x31))
    return features

In [ ]:
def all_features(t):
    features = {}
    for i in t:
        x1=features_tfidf[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in features_tfidf.keys() else np.zeros(len(features_tfidf['0|340485']))
        x2=features_fasttext[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in features_fasttext.keys() else np.zeros(len(features_fasttext['0|6760']))
        
        x4=features_hosts[str(i[1])] if str(i[1]) in features_hosts.keys() else np.zeros(len(features_hosts['10059']))
        x5=features_docs[str(i[1])] if str(i[1]) in features_docs.keys() else np.zeros(len((features_docs['100059'])))
        x6=qdocs_sDBN[str(i[0])+"|"+str(i[1])]  if str(i[0])+"|"+str(i[1]) in qdocs_sDBN.keys() else np.zeros(3)
        x7=qhost_sDBN[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in qhost_sDBN.keys() else np.zeros(3)
        x8=docs_sDBN[str(i[1])] if str(i[1]) in docs_sDBN.keys() else np.zeros(3)
        x9=host_sDBN[str(i[1])] if str(i[1]) in host_sDBN.keys() else np.zeros(3)
        x10=features_qhosts[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in features_qhosts.keys() else np.zeros(len((features_docs['100059'])))
        x11=features_qdocs[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in features_qdocs.keys() else np.zeros(len((features_docs['100059'])))
        x12=features_queries[str(i[0])] if str(i[0]) in features_queries.keys() else np.zeros(6)        
        
        x14=features_bert[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in features_bert.keys() else np.zeros(len(features_bert['0|340485']))
        x15=features_transforms[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in features_transforms.keys() else np.zeros(len(features_transforms['0|340485']))
        try:
            x21 = dvs[(str(i[0]), str(i[1]))]
        except:
            x21 = np.array([0.0])
        try:    
            x22 = trft[(str(i[0]), str(i[1]))]
        except:
            x22 = np.array([0.0])
        try:    
            x23 = muse[(str(i[0]), str(i[1]))]
        except:
            x23 = np.array([0.0])
        try:
            x24 = past[(str(i[0]), str(i[1]))]
        except:
            x24 = np.array([0.0])
        try:
            x30 = quse[(str(i[0]), str(i[1]))]
        except:
            x30 = np.array([0.0])         
        try:
            x25 = hf[hostdict[str(i[1])]]
        except:

            [x25] = [[0.0] * 13]
        try:
            x26 = qhf[(str(i[0]),hostdict[str(i[1])])]
        except:

            [x26] = [[0.0] * 43]

        try:
            x27 = uf[str(i[1])]
        except:

            [x27] = [[0.0] * 13]

        try:
            x28 = quf[(str(i[0]), str(i[1]))]
        except:

            [x28] = [[0.0] * 43]
        try:
            x29 = dbndoc[str(i[1])]
        except:

            [x29] = [[0.0] * 3]
            
            
        features[str(i[0])+"|"+str(i[1])+"|"+str(i[2])]=np.hstack((x1,x2,x4,x5,x6,x7,x8,x9,x10,x11,x12,x14,x15, x21, x22, x23, x24))

        
        #features[str(i[0])+"|"+str(i[1])+"|"+str(i[2])]=np.hstack((x1,x2,x14,x15, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30))
    return features

In [ ]:
def all_features(t):
    features = {}
    for i in t:
        x1=features_tfidf[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in features_tfidf.keys() else np.zeros(len(features_tfidf['0|340485']))
        x2=features_fasttext[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in features_fasttext.keys() else np.zeros(len(features_fasttext['0|6760']))
                
        
        x14=features_bert[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in features_bert.keys() else np.zeros(len(features_bert['0|340485']))
        x15=features_transforms[str(i[0])+"|"+str(i[1])] if str(i[0])+"|"+str(i[1]) in features_transforms.keys() else np.zeros(len(features_transforms['0|340485']))
        try:
            x21 = dvs[(str(i[0]), str(i[1]))]
        except:
            x21 = np.array([0.0])
        try:    
            x22 = trft[(str(i[0]), str(i[1]))]
        except:
            x22 = np.array([0.0])
        try:    
            x23 = muse[(str(i[0]), str(i[1]))]
        except:
            x23 = np.array([0.0])
        try:
            x24 = past[(str(i[0]), str(i[1]))]
        except:
            x24 = np.array([0.0])
        try:
            x30 = quse[(str(i[0]), str(i[1]))]
        except:
            x30 = np.array([0.0])
        try:
            x31 = texttf[(str(i[0]), str(i[1]))]
        except:
            x31 = np.array([0.0])
        try:
            x32 = newbm[(str(i[0]), str(i[1]))]
        except:
            x32 = np.array([0.0])            
        try:
            x25 = hf[hostdict[str(i[1])]]
        except:

            [x25] = [[0.0] * 13]
        try:
            x26 = qhf[(str(i[0]),hostdict[str(i[1])])]
        except:

            [x26] = [[0.0] * 43]

        try:
            x27 = uf[str(i[1])]
        except:

            [x27] = [[0.0] * 13]

        try:
            x28 = quf[(str(i[0]), str(i[1]))]
        except:

            [x28] = [[0.0] * 43]
        try:
            x29 = dbndoc[str(i[1])]
        except:

            [x29] = [[0.0] * 3]
            
            
        #features[str(i[0])+"|"+str(i[1])+"|"+str(i[2])]=np.hstack((x1,x2,x4,x5,x6,x7,x8,x9,x10,x11,x12,x14,x15, x21, x22, x23, x24))

        
        features[str(i[0])+"|"+str(i[1])+"|"+str(i[2])]=np.hstack((x1,x2,x14,x15, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30, x31, x32))
    return features

In [ ]:
file =open("normal_queries.txt" ,'r', encoding='utf-8')
queries = {}
for line in file.readlines():
    l=line.split('\t')
    queries[int(l[0])]= l[1].strip()
file.close()

In [ ]:
file =open("qspell.txt" ,'r', encoding='utf-8')
queries_not_normal = {}
for line in file.readlines():
    l=line.split('\t')
    queries_not_normal[int(l[0])]= l[1].strip()
file.close()

In [ ]:
file =open("title.txt" ,'r', encoding='utf-8')
titles_not_normal= {}
for line in file.readlines():
    l=line.split('\t')
    titles_not_normal[int(l[0])]= l[1].strip()
file.close()

In [ ]:
file =open("normal_titles.txt" ,'r', encoding='utf-8')
titles= {}
for line in file.readlines():
    l=line.split('\t')
    titles[int(l[0])]= l[1].strip()
file.close()

In [ ]:
file = open("train.marks.tsv/train.marks.tsv", 'r')
train=[]
train_map={}
queries_doc_map_all={}
marks={}
for l in tqdm.tqdm(file.readlines()):
    splits=l.split('\t')
    if int(splits[2])==0:
        continue
    train.append([int(splits[0]),int(splits[1]), int(splits[2])])
    marks[splits[0]+"|"+splits[1]]=int(splits[2])
    train_map[splits[0]+'|'+splits[1]]=splits[2].strip()
    if int(splits[0]) not in queries_doc_map_all.keys():
        queries_doc_map_all[int(splits[0])]=[int(splits[1])]
    else:
        queries_doc_map_all[int(splits[0])].append(int(splits[1]))
train=np.array(train)
file.close()
test_map={}
file = open("sample.csv/sample.csv", 'r')
file.readline()
test=[]
for l in tqdm.tqdm(file.readlines()):
    splits=l.split(',')
    test.append([int(splits[0]),int(splits[1]), -1])
    test_map[splits[0]+'|'+splits[1].strip()]="-1"
    if int(splits[0]) not in queries_doc_map_all.keys():
        queries_doc_map_all[int(splits[0])]=[int(splits[1])]
    else:
        queries_doc_map_all[int(splits[0])].append(int(splits[1]))
file.close()

In [ ]:
features_train=all_features(train)
features_test=all_features(test)

In [ ]:
from scipy.sparse import csr_matrix

def make_data(features):
    qid_data=[]
    y_data = []
    doc_ids=[]
    X_data=[]
    for l in features.keys():
        line=l.split("|")
        qid_data.append(int(line[0]))
        y_data.append(float(line[2]))
        doc_ids.append(int(line[1]))
        X_data.append(features[l])
    qid_data=np.array(qid_data)
    y_data=np.array(y_data)
    X_data= csr_matrix(X_data)
    doc_ids= np.array(doc_ids)
    group_data =np.unique(qid_data, return_counts=True)[1]
    return X_data, y_data, qid_data, group_data, doc_ids

In [ ]:
X_train,y_train, qid_train, group_train, doc_ids_train = make_data(features_train)
X_test,y_test, qid_test, group_test, doc_ids_test = make_data(features_test)

In [ ]:
import lightgbm as gbm

params={'boosting_type':'gbdt','score':'ndcg@5',
'n_estimators':100,
'subsample':0.9,
'learning_rate':0.1,
'num_leaves':63,}
Ranker1 = gbm.LGBMRanker(**params)
Ranker1.fit(X_train, y_train, group=group_train, verbose=1)

In [ ]:
params={'boosting_type':'gbdt','score':'ndcg@5',
'n_estimators':500,
'subsample':0.9,
'learning_rate':0.1,
'num_leaves':127,}
Ranker2 = gbm.LGBMRanker(**params)
Ranker2.fit(X_train, y_train, group=group_train, verbose=1)

In [ ]:
params={'boosting_type':'gbdt','score':'ndcg@5',
'n_estimators':1000,
'subsample':0.95,
'learning_rate':0.01,
'num_leaves':31,}
Ranker3 = gbm.LGBMRanker(**params)
Ranker3.fit(X_train, y_train, group=group_train, verbose=1)


In [ ]:
params={'boosting_type':'gbdt','score':'ndcg@5',
'n_estimators':3000,
'subsample':0.95,
'learning_rate':0.007,
'num_leaves':29,}
Ranker = gbm.LGBMRanker(**params)
Ranker.fit(X_train, y_train, group=group_train, verbose=1)

In [ ]:
pred =(Ranker.predict(X_test)+Ranker1.predict(X_test)+Ranker2.predict(X_test)+Ranker3.predict(X_test))


In [ ]:
import copy

def save_submission(pred_qids, preds, all_doc_ids, filename):
    fout = open(filename, 'w')
    fout.write('QueryId,DocumentId\n')
    for qid in np.unique(pred_qids):
        q_doc_idxs = np.argwhere(pred_qids == qid).ravel()
        #print(q_doc_idxs)
        doc_ids = copy.deepcopy(all_doc_ids[q_doc_idxs]).ravel()
        #print(doc_ids)
        q_doc_scores = preds[q_doc_idxs]
        #print(q_doc_scores)
        sorted_doc_ids = doc_ids[np.argsort(q_doc_scores)[::-1]]
        for did in sorted_doc_ids:
            fout.write('{0},{1}\n'.format(qid, did))
        
    fout.close

In [ ]:
save_submission(qid_test,pred,doc_ids_test, 'test8.txt')


In [ ]:
import lightgbm as gbm
lgrank = gbm.LGBMRanker(objective='lambdarank', max_depth=12, n_estimators= 1000, subsample=0.8, learning_rate=0.01, num_leaves=64)

In [ ]:
%%time
lgrank.fit(X_train, y_train, group=group_train)

In [ ]:
pred = lgrank.predict(X_test)

In [ ]:
save_submission(qid_test,pred,doc_ids_test, 'test9.txt')


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

embed3 = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [ ]:
from deeppavlov import build_model
from deeppavlov import configs
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder
from tqdm import tqdm

In [ ]:
pip install deeppavlov

In [ ]:
name, alias = "elmo_ru_news", "news"
model = ELMoEmbedder(str(prefix / "ELMo/{}".format(name)))

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

embed3 = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")


In [ ]:
module3 = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')
module4= hub.load('https://tfhub.dev/google/universal-sentence-encoder-qa/3')

In [ ]:
def sim_USE(qid):
    questions = [queries[qid]]
    question_embeddings = module3.signatures['question_encoder'](
            tf.constant(questions))
    responses = list(generator_titles(queries_doc_map_all[qid]))
    response_contexts = list(generator_doc(qid))
    response_embeddings = module3.signatures['response_encoder'](
        input=tf.constant(responses),
        context=tf.constant(response_contexts))
    sim1 = np.inner(question_embeddings['outputs'], response_embeddings['outputs']).ravel()
    questions = [tr_queries[qid]]
    responses = list(generator_titles_tr(queries_doc_map_all[qid]))
    response_contexts = list(generator_titles_tr(queries_doc_map_all[qid]))
    question_embeddings = module4.signatures['question_encoder'](
            tf.constant(questions))
    response_embeddings = module4.signatures['response_encoder'](
        input=tf.constant(responses),
        context=tf.constant(response_contexts))
    sim4 = np.inner(question_embeddings['outputs'], response_embeddings['outputs']).ravel()
    sim5 = cosine(embed3([tr_queries[qid]])[0].numpy().reshape(1,-1), embed3(list(generator_titles_tr(queries_doc_map_all[qid]))).numpy()).ravel()
    return np.hstack((sim1.reshape(-1,1), sim4.reshape(-1,1), sim5.reshape(-1,1)))